In [307]:
import sys
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px

In [308]:
events = pd.read_pickle("../dash_v1/event_filtered.pickle")

In [309]:
events[:1]

,Unnamed: 0,event_id,event_start,event_area,event_length,event_si,date,area,lat,latMax,lon,lonMax,maxPrec,meanPre,si,size,stdv,country,event_year,event_month
0,34,199900024,1999-01-01 06:00:00,0.97,9,0.02,1999-01-01 06:00:00,0.03,37.490002,37.540001,10.84,10.83,9.13,8.3,0.0,2,0.82,INT,1999,1


In [310]:
avg_si = events.groupby(['event_year'])['event_si'].mean()
avg_si = pd.DataFrame(avg_si)
avg_si['event_year'] = avg_si.index
avg_si.reset_index(drop=True,inplace=True)

In [311]:
max_si = events.groupby(['event_year'])['event_si'].max()
max_si = pd.DataFrame(max_si)
max_si['event_year'] = max_si.index
max_si.reset_index(drop=True,inplace=True)

In [312]:
events['day'] = [datetime.datetime.strptime(e, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') for e in events.date]

events_mean = events.groupby(['day'])['event_si'].mean()
events_mean = pd.DataFrame(events_mean)

events_mean['date'] = events_mean.index 
events_mean.reset_index(drop=True,inplace=True)

In [313]:
x = events_mean.date
y = events_mean.event_si

x_avg = avg_si.event_year
y_avg = avg_si.event_si

x_max = max_si.event_year
y_max = max_si.event_si

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,name='per event'))
fig.add_trace(go.Scatter(x=x_avg, y=y_avg,name='average per year'))
fig.add_trace(go.Scatter(x=x_max, y=y_max,name='max per year'))
fig.update_layout(title_text="Severity Index")
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Severity')
fig.show()

5 years

In [314]:
events['year_cluster'] = events.event_year - events.event_year % 5
avg_si = events.groupby(['year_cluster'])['event_si'].mean()
avg_si = pd.DataFrame(avg_si)
avg_si['year_cluster'] = avg_si.index
avg_si.reset_index(drop=True,inplace=True)
avg_si = avg_si[:-1]
avg_si = avg_si.drop(avg_si.index[0])

In [315]:
max_si = events.groupby(['year_cluster'])['event_si'].max()
max_si = pd.DataFrame(max_si)
max_si['year_cluster'] = max_si.index
max_si.reset_index(drop=True,inplace=True)
max_si = max_si[:-1]
max_si = max_si.drop(max_si.index[0])

In [316]:
x = events_mean.date
y = events_mean.event_si

x_avg = avg_si.year_cluster
y_avg = avg_si.event_si

x_max = max_si.year_cluster
y_max = max_si.event_si

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,name='per event'))
fig.add_trace(go.Scatter(x=x_avg, y=y_avg,name='average per 5 years'))
fig.add_bar(x=x_max, y=y_max,name='max per 5 years')
fig.update_layout(title_text="Severity Index")
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Severity')
fig.show()

In [317]:
# events per year
events_per_year = events.groupby('event_year')['event_id'].nunique().reset_index()
events_mean = events_per_year.event_id.mean()

In [318]:
events_filtered = events.drop_duplicates(subset=['event_id'], keep='first')
events_per_cluster = events_filtered.groupby('year_cluster')['event_id'].count()
events_per_cluster = pd.DataFrame(events_per_cluster)
events_per_cluster['avg_per_cluster'] = events_per_cluster.event_id / 5
events_per_cluster.reset_index(inplace=True)
events_per_cluster = events_per_cluster[:-1]
events_per_cluster = events_per_cluster.drop(events_per_cluster.index[0])

In [319]:
fig = go.Figure()
fig.add_bar(x=events_per_year.event_year, y=events_per_year.event_id ,name='per event')
fig.add_trace(go.Scatter(x=events_per_cluster.year_cluster,
                         y=events_per_cluster.avg_per_cluster,
                         name='average per 5 years'))
fig.add_shape(type='line',
                x0=1979,
                y0=events_mean,
                x1=2017,
                y1=events_mean,
                line=dict(color='black',),
                xref='x',
                yref='y',
              name='Average overall'
)
fig.update_layout(title_text="Events per year")
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Events')
fig.show()


In [320]:
# events per month
events_per_month = events.groupby('event_month')['event_id'].nunique().reset_index()
events_mean_month = events_per_month.event_id.mean()
max_month = events.groupby(['year_cluster'])['event_si'].max()

In [321]:
fig = go.Figure()
fig.add_bar(x=events_per_month.event_month, y=events_per_month.event_id ,name='per event')
fig.add_shape(type='line',
                x0=1,
                y0=events_mean_month,
                x1=12,
                y1=events_mean_month,
                line=dict(color='black',),
                xref='x',
                yref='y',
              name='Average overall'
)
fig.update_layout(title_text="Events per Month")
fig.update_xaxes(title_text='Month')
fig.update_yaxes(title_text='Events')
fig.show()



In [322]:
avg_duration = events.groupby(['event_year'])['event_length'].mean()
avg_duration = pd.DataFrame(avg_duration)
avg_duration['event_year'] = avg_duration.index
avg_duration.reset_index(drop=True,inplace=True)

In [323]:
max_duration = events.groupby(['event_year'])['event_length'].max()
max_duration = pd.DataFrame(max_duration)
max_duration['event_year'] = max_duration.index
max_duration.reset_index(drop=True,inplace=True)

In [324]:
events['day'] = [datetime.datetime.strptime(e, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') for e in events.date]

events_mean = events.groupby(['day'])['event_length'].mean()
events_mean = pd.DataFrame(events_mean)

events_mean['date'] = events_mean.index
events_mean.reset_index(drop=True,inplace=True)

In [325]:
x = events_mean.date
y = events_mean.event_length

x_avg = avg_duration.event_year
y_avg = avg_duration.event_length

x_max = max_duration.event_year
y_max = max_duration.event_length

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,name='per event'))
fig.add_trace(go.Scatter(x=x_avg, y=y_avg,name='average per year'))
fig.add_trace(go.Scatter(x=x_max, y=y_max,name='max per year'))
fig.update_layout(title_text="Duration Index")
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Duration in hours')
fig.show()